# Step 4: Make the forecast

In this notebook we will prepare the dataset needed to create the forecast. We will build the multi-step forecast using recursive forecasting.

In [1]:
import datetime
from pathlib import Path

import joblib
import numpy as np
import pandas as pd

from sklearn import set_config
set_config(transform_output="pandas")

# Config paths

Specify the paths and directories from which we will read and write data to. In practice we would store this in a separate file rather than duplicate it across all notebooks. For simplicity, we specify the paths in the notebook itself.

In [2]:
# Directory containing the raw data
data_sources = Path("../data_sources")

# Output processed data (i.e., the base dataset)
processed_data_dir = Path("../processed_data")

# Artifacts directory for storing the
# training data, models, pipelines etc.
artifacts_dir = Path("../artifacts")
training_dir = artifacts_dir / "training"
pipeline_dir = artifacts_dir / "pipeline"
model_dir = artifacts_dir / "model"
forecast_dir = artifacts_dir / "forecast"

# Load model

In [3]:
model = joblib.load(model_dir / "model.joblib")

# Load feature engineering pipeline

Ensure that we have run (present at the start of the notebook): 
    
```Python
from sklearn import set_config
set_config(transform_output="pandas")
```

Otherwise, the pipeline may output a numpy array rather than a pandas dataframe.

In [4]:
pipeline = joblib.load(pipeline_dir/"pipeline.joblib")

# Set forecast parameters

In [5]:
forecast_start_date = pd.to_datetime("2016-05-23")
num_steps = 14
forecast_horizon = pd.date_range(start=forecast_start_date, 
                                 periods=num_steps, 
                                 freq="D")

# Specify how much data before `forecast_start_date` is required
# for feature engineering.
look_back_window_size = np.timedelta64(31, "D")

# Create the predict dataset

We're preparing a dataframe to enable us to implement the following loop.

![](./images/recursive_forecasting/Slide1.png)
![](./images/recursive_forecasting/Slide2.png)
![](./images/recursive_forecasting/Slide3.png)
![](./images/recursive_forecasting/Slide4.png)

Query a subset of base dataset and prepare future values.

In [6]:
# Read the base data set of just one store.
# Read only the time period needed.
f_in = processed_data_dir / "data"
df = pd.read_parquet(
    path=f_in, 
    engine="pyarrow", 
    filters=[("store_id", "=", "CA_1"),
             ("date", ">=", forecast_start_date-look_back_window_size), 
             ("date", "<", forecast_start_date ) 
            ]
)


# Remove unused categories.
df["id"] = df["id"].cat.remove_unused_categories()

# Set time series ID and date as index.
# This is needed for some sktime transformers.
df = df.set_index(["id", "date"]).sort_index()
df.head()

item_id    dept_id   cat_id  \
id                            date                                            
HOBBIES_1_001_CA_1_evaluation 2016-04-22  HOBBIES_1_001  HOBBIES_1  HOBBIES   
                              2016-04-23  HOBBIES_1_001  HOBBIES_1  HOBBIES   
                              2016-04-24  HOBBIES_1_001  HOBBIES_1  HOBBIES   
                              2016-04-25  HOBBIES_1_001  HOBBIES_1  HOBBIES   
                              2016-04-26  HOBBIES_1_001  HOBBIES_1  HOBBIES   

                                         state_id  y event_name_1  \
id                            date                                  
HOBBIES_1_001_CA_1_evaluation 2016-04-22       CA  0     no_event   
                              2016-04-23       CA  1     no_event   
                              2016-04-24       CA  1     no_event   
                              2016-04-25       CA  0     no_event   
                              2016-04-26       CA  0     no_event   

                                         event_type_1 event_name_2  \
id                            date                                   
HOBBIES_1_001_CA_1_evaluation 2016-04-22     no_event     no_event   
                              2016-04-23     no_event     no_event   
                              2016-04-24     no_event     no_event   
                              2016-04-25     no_event     no_event   
                              2016-04-26     no_event     no_event   

                                         event_type_2  snap_CA  snap_TX  \
id                            date                                        
HOBBIES_1_001_CA_1_evaluation 2016-04-22     no_event        0        0   
                              2016-04-23     no_event        0        0   
                              2016-04-24     no_event        0        0   
                              2016-04-25     no_event        0        0   
                              2016-04-26     no_event        0        0   

                                          snap_WI  sell_price store_id  
id                            date                                      
HOBBIES_1_001_CA_1_evaluation 2016-04-22        0        8.38     CA_1  
                              2016-04-23        0        8.38     CA_1  
                              2016-04-24        0        8.38     CA_1  
                              2016-04-25        0        8.38     CA_1  
                              2016-04-26        0        8.38     CA_1

Create the predict dataset by extending the index and filling in future values. We will naively project the last value carried forward for each non-null column.

In [7]:
predict_data_time = pd.date_range(
    start=forecast_start_date - look_back_window_size,
    end=forecast_horizon[-1],
    freq="D",
)

In [8]:
ts_ids = df.index.levels[0]
new_idx = pd.MultiIndex.from_product([ts_ids, 
                                      predict_data_time], 
                                     names=df.index.names)

df_predict = pd.DataFrame(data=df,
                          index=new_idx, 
                         )

We naively project the last value carried forward for each feature for all time series into the forecast horizon. We could generate different forecasting scenarios by manipulating the future values of our variables here (e.g., `sell_price` or the `snap_` variables), for simplicity, we just carry the last value forward into the forecasting horizon (this could be problematic for the `event_` variables, we don't want to project every future day as Christmas day).

In [9]:
df_predict = df_predict.groupby(level=0).fillna(method="ffill")

Remove any values of y during the forecast horizon. Just for sanity, not really required.

In [10]:
time_index = df_predict.index.get_level_values(-1)
time_mask = (time_index >= forecast_horizon[0]) & (time_index <= forecast_horizon[-1])
df_predict.loc[time_mask, ["y"]] = np.NaN

Check that our feature engineering pipeline is functioning as expected.

In [11]:
pipeline.transform(df_predict.head())

year  month_of_year  week_of_year  \
id                            date                                            
HOBBIES_1_001_CA_1_evaluation 2016-04-22   1.0       0.272727      0.288462   
                              2016-04-23   1.0       0.272727      0.288462   
                              2016-04-24   1.0       0.272727      0.288462   
                              2016-04-25   1.0       0.272727      0.307692   
                              2016-04-26   1.0       0.272727      0.307692   

                                          week_of_month  day_of_month  \
id                            date                                      
HOBBIES_1_001_CA_1_evaluation 2016-04-22           0.75      0.700000   
                              2016-04-23           0.75      0.733333   
                              2016-04-24           0.75      0.766667   
                              2016-04-25           0.75      0.800000   
                              2016-04-26           0.75      0.833333   

                                          day_of_week  is_weekend  \
id                            date                                  
HOBBIES_1_001_CA_1_evaluation 2016-04-22     0.666667         0.0   
                              2016-04-23     0.833333         1.0   
                              2016-04-24     1.000000         1.0   
                              2016-04-25     0.000000         0.0   
                              2016-04-26     0.166667         0.0   

                                          time_since_2012-01-01 00:00:00  \
id                            date                                         
HOBBIES_1_001_CA_1_evaluation 2016-04-22                        0.981285   
                              2016-04-23                        0.981909   
                              2016-04-24                        0.982533   
                              2016-04-25                        0.983157   
                              2016-04-26                        0.983780   

                                           y_lag_1   y_lag_2  ...  \
id                            date                            ...   
HOBBIES_1_001_CA_1_evaluation 2016-04-22  0.004630  0.001543  ...   
                              2016-04-23  0.000000  0.004630  ...   
                              2016-04-24  0.001543  0.000000  ...   
                              2016-04-25  0.001543  0.001543  ...   
                              2016-04-26  0.000000  0.001543  ...   

                                          sell_price_lag_7  sell_price_lag_14  \
id                            date                                              
HOBBIES_1_001_CA_1_evaluation 2016-04-22          0.270262           0.266387   
                              2016-04-23          0.270262           0.270262   
                              2016-04-24          0.270262           0.270262   
                              2016-04-25          0.270262           0.270262   
                              2016-04-26          0.270262           0.270262   

                                          sell_price_lag_28  \
id                            date                            
HOBBIES_1_001_CA_1_evaluation 2016-04-22           0.266387   
                              2016-04-23           0.266387   
                              2016-04-24           0.266387   
                              2016-04-25           0.266387   
                              2016-04-26           0.266387   

                                          sell_price_mean_1_7  \
id                            date                              
HOBBIES_1_001_CA_1_evaluation 2016-04-22             0.270262   
                              2016-04-23             0.270262   
                              2016-04-24             0.270262   
                              2016-04-25             0.270262   
                              2016-04-26             0.270262   

         

# Make forecast

Now that we have:
* a trained one-step ahead forecasting model `model`;
* the feature engineering pipeline to recursively computer our features `pipeline`;
* the predict dataset that contains the future values of our features `df_predict`;

we can build our multi-step forecast using recursive forecasting.

In [12]:
# --- RECURSIVE FORECASTING LOOP --- #
for forecast_time in forecast_horizon:    
    # Compute features during the forecast horizon
    X_test = pipeline.transform(df_predict)
    time_mask = X_test.index.get_level_values(-1) == forecast_time
    X_test_ = X_test.loc[time_mask] 

    # Predict one step ahead. 
    y_pred = model.predict(X_test_)
    
    # Append forecast to the target variable columnn in our
    # dynamic forecast dataframe `df_predict`. This `df_predict`
    # is ready for the next iteration where we will re-compute
    # features derived from the target such as lags and windows.
    df_predict.loc[time_mask, ["y"]] = y_pred

In [13]:
# --- GET FORECAST FROM PREDICT DATAFRAME--- #    
time_mask = (df_predict.index.get_level_values(-1) >= forecast_horizon[0]) & (
    df_predict.index.get_level_values(-1) <= forecast_horizon[-1]
)

df_predict["is_forecast"] = time_mask
y_pred = df_predict.loc[time_mask, ["y"]]

Our outputs are:

 - `df_predict`: contains the actual values of `y` before the forecast horizon and contains the forecasted values of `y` after the forecast horizon. It also contains the original variables used to build our features.

 - `y_pred`: contains just our forecasted values of `y`.

- `X_test`: contains the actual feature values passed to `model.predict` during the forecast horizon.

Let's check the predict dataframe, `df_predict`, and the dataframe with our forecasts, `y_pred`.

In [14]:
display(df_predict.head(), y_pred.head())

item_id    dept_id   cat_id  \
id                            date                                            
HOBBIES_1_001_CA_1_evaluation 2016-04-22  HOBBIES_1_001  HOBBIES_1  HOBBIES   
                              2016-04-23  HOBBIES_1_001  HOBBIES_1  HOBBIES   
                              2016-04-24  HOBBIES_1_001  HOBBIES_1  HOBBIES   
                              2016-04-25  HOBBIES_1_001  HOBBIES_1  HOBBIES   
                              2016-04-26  HOBBIES_1_001  HOBBIES_1  HOBBIES   

                                         state_id    y event_name_1  \
id                            date                                    
HOBBIES_1_001_CA_1_evaluation 2016-04-22       CA  0.0     no_event   
                              2016-04-23       CA  1.0     no_event   
                              2016-04-24       CA  1.0     no_event   
                              2016-04-25       CA  0.0     no_event   
                              2016-04-26       CA  0.0     no_event   

                                         event_type_1 event_name_2  \
id                            date                                   
HOBBIES_1_001_CA_1_evaluation 2016-04-22     no_event     no_event   
                              2016-04-23     no_event     no_event   
                              2016-04-24     no_event     no_event   
                              2016-04-25     no_event     no_event   
                              2016-04-26     no_event     no_event   

                                         event_type_2  snap_CA  snap_TX  \
id                            date                                        
HOBBIES_1_001_CA_1_evaluation 2016-04-22     no_event      0.0      0.0   
                              2016-04-23     no_event      0.0      0.0   
                              2016-04-24     no_event      0.0      0.0   
                              2016-04-25     no_event      0.0      0.0   
                              2016-04-26     no_event      0.0      0.0   

                                          snap_WI  sell_price store_id  \
id                            date                                       
HOBBIES_1_001_CA_1_evaluation 2016-04-22      0.0        8.38     CA_1   
                              2016-04-23      0.0        8.38     CA_1   
                              2016-04-24      0.0        8.38     CA_1   
                              2016-04-25      0.0        8.38     CA_1   
                              2016-04-26      0.0        8.38     CA_1   

                                          is_forecast  
id                            date                     
HOBBIES_1_001_CA_1_evaluation 2016-04-22        False  
                              2016-04-23        False  
                              2016-04-24        False  
                              2016-04-25        False  
                              2016-04-26        False

y
id                            date                
HOBBIES_1_001_CA_1_evaluation 2016-05-23  1.045888
                              2016-05-24  0.912308
                              2016-05-25  0.939688
                              2016-05-26  1.009833
                              2016-05-27  1.104322

Let's save the forecast and features over the forecast horizon.

In [15]:
f_out = forecast_dir / "forecast.parquet"
y_pred.to_parquet(f_out)

f_out = forecast_dir / "predict_dataset.parquet"
df_predict.to_parquet(f_out)

f_out = forecast_dir / "predict_dataset_features.parquet"
X_test.to_parquet(f_out)